## Equations for EKF
These equations came from https://www.jhuapl.edu/Content/techdigest/pdf/V31-N02/31-02-Barton.pdf


In [1]:
from sympy import symbols, Matrix, init_printing, pprint, zeros
init_printing(True)

Define symbols for the quaternion, state and the measurements.
For this experiment we will only be using Gyroscope and Accelerometer as inputs and the state vector also contains the bias of the Gyroscope.

In [2]:
qw, qx, qy, qz = symbols('qw qx qy qz')
gx, gy, gz = symbols('gx gy gz')
ax, ay, az = symbols('ax ay az')
wbx, wby, wbz = symbols('wbx wby wbz')

Define measurement vectors now utilizing the above defined symbols.
Note that the Measurement vectors are always matrices and thats how they are also 
represented in the code.

In [3]:
gyro = Matrix([[gx], [gy], [gz]])
bias = Matrix([[wbx], [wby], [wbz]])

In [4]:
accel = Matrix([[ax], [ay], [az]])

Since our state vector is just a quaternion, the $\dot{x}$ is just the derivative of the quaternion.
https://fgiesen.wordpress.com/2012/08/24/quaternion-differentiation/. 
The derivative of Quaternion will be 

\begin{equation*}
q'(t) = 0.5 * q(t) * \omega(t)\\

q'(t) = 0.5 *
\begin{bmatrix}
0 & -\omega_x & -\omega_y & -\omega_z\\
\omega_x & 0  & \omega_z  & -\omega_y\\
\omega_y & -\omega_z & 0 & \omega_x \\
\omega_z & \omega_x & -\omega_y & 0 \\
\end{bmatrix}
*
\begin{bmatrix}
q_w\\
q_x\\
q_y\\
q_z
\end{bmatrix}

\end{equation*}

The above equation can be manipulated to form

\begin{equation*}
q'(t) = 0.5 * 
\begin{bmatrix}
-q_x & -q_y & -q_z\\
q_w & -q_z & q_y\\
q_z & q_w & -q_x\\
-q_y & q_x & q_w
\end{bmatrix}
*
\begin{bmatrix}
\omega_x\\
\omega_y\\
\omega_z
\end{bmatrix}

\end{equation*}

The above is a left quaternion multiplication where the right will be

\begin{equation*}
q'(t) = 0.5 * 
\begin{bmatrix}
-q_x & -q_y & -q_z\\
q_w & q_z & -q_y\\
-q_z & q_w & q_x\\
q_y & -q_x & q_w
\end{bmatrix}
*
\begin{bmatrix}
\omega_x\\
\omega_y\\
\omega_z
\end{bmatrix}

\end{equation*}


Define our state vector. In our case as of now, the state vector will just be a quaternion as we are only interested in the orientation of pixhawk. We will use Gyroscope measurement as input and Accelerometer as the measurement to EKF. 

In [5]:
state = Matrix([[qw], [qx], [qy], [qz], [wbx], [wby], [wbz]])

In [6]:
omega2qdot = 0.5 * Matrix([[-qx, -qy, -qz],
                            [qw, qz, -qy],
                            [-qz, qw, qx],
                            [qy, -qx, qw]])

calculate the derivative matrix that contains the derivaties of the elements in state vector. The bias is assumed constant and its derivative will be zero.

In [7]:
jacAngVel = omega2qdot * (gyro - bias)
xdot = zeros(7, 1)
xdot[0, 0] = jacAngVel[0]
xdot[1, 0] = jacAngVel[1]
xdot[2, 0] = jacAngVel[2]
xdot[3, 0] = jacAngVel[3]
pprint(xdot)

⎡-0.5⋅qx⋅(gx - wbx) - 0.5⋅qy⋅(gy - wby) - 0.5⋅qz⋅(gz - wbz)⎤
⎢                                                          ⎥
⎢0.5⋅qw⋅(gx - wbx) - 0.5⋅qy⋅(gz - wbz) + 0.5⋅qz⋅(gy - wby) ⎥
⎢                                                          ⎥
⎢0.5⋅qw⋅(gy - wby) + 0.5⋅qx⋅(gz - wbz) - 0.5⋅qz⋅(gx - wbx) ⎥
⎢                                                          ⎥
⎢0.5⋅qw⋅(gz - wbz) - 0.5⋅qx⋅(gy - wby) + 0.5⋅qy⋅(gx - wbx) ⎥
⎢                                                          ⎥
⎢                            0                             ⎥
⎢                                                          ⎥
⎢                            0                             ⎥
⎢                                                          ⎥
⎣                            0                             ⎦


We use the above equation to propagate the state for EKF.

$\begin{array}{l|l}
\text{linear Kalman filter} & \text{EKF} \\
\hline 
& \boxed{\mathbf F = {\frac{\partial{f(\mathbf x_t, \mathbf u_t)}}{\partial{\mathbf x}}}\biggr|_{{\mathbf x_t},{\mathbf u_t}}} \\
\mathbf{\bar x} = \mathbf{Fx} + \mathbf{Bu} & \boxed{\mathbf{\bar x} = f(\mathbf x, \mathbf u)}  \\
\mathbf{\bar P} = \mathbf{FPF}^\mathsf{T}+\mathbf Q  & \mathbf{\bar P} = \mathbf{FPF}^\mathsf{T}+\mathbf Q \\
\hline
& \boxed{\mathbf H = \frac{\partial{h(\bar{\mathbf x}_t)}}{\partial{\bar{\mathbf x}}}\biggr|_{\bar{\mathbf x}_t}} \\
\textbf{y} = \mathbf z - \mathbf{H \bar{x}} & \textbf{y} = \mathbf z - \boxed{h(\bar{x})}\\
\mathbf{K} = \mathbf{\bar{P}H}^\mathsf{T} (\mathbf{H\bar{P}H}^\mathsf{T} + \mathbf R)^{-1} & \mathbf{K} = \mathbf{\bar{P}H}^\mathsf{T} (\mathbf{H\bar{P}H}^\mathsf{T} + \mathbf R)^{-1} \\
\mathbf x=\mathbf{\bar{x}} +\mathbf{K\textbf{y}} & \mathbf x=\mathbf{\bar{x}} +\mathbf{K\textbf{y}} \\
\mathbf P= (\mathbf{I}-\mathbf{KH})\mathbf{\bar{P}} & \mathbf P= (\mathbf{I}-\mathbf{KH})\mathbf{\bar{P}}
\end{array}$

We don't normally use $\mathbf{Fx}$ to propagate the state for the EKF as the linearization causes inaccuracies. It is typical to compute $\bar{\mathbf x}$ using a suitable numerical integration technique such as Euler or Runge Kutta. Thus I wrote $\mathbf{\bar x} = f(\mathbf x, \mathbf u)$. For the same reasons we don't use $\mathbf{H\bar{x}}$ in the computation for the residual, opting for the more accurate $h(\bar{\mathbf x})$.

Euler integration will have the form $\bar{x} = \bar{x} + \dot{x} * \delta{t}$

In [8]:
f_t = xdot.jacobian(state)
pprint(f_t)

⎡       0          -0.5⋅gx + 0.5⋅wbx  -0.5⋅gy + 0.5⋅wby  -0.5⋅gz + 0.5⋅wbz  0.
⎢                                                                             
⎢0.5⋅gx - 0.5⋅wbx          0          -0.5⋅gz + 0.5⋅wbz  0.5⋅gy - 0.5⋅wby   -0
⎢                                                                             
⎢0.5⋅gy - 0.5⋅wby  0.5⋅gz - 0.5⋅wbz           0          -0.5⋅gx + 0.5⋅wbx  0.
⎢                                                                             
⎢0.5⋅gz - 0.5⋅wbz  -0.5⋅gy + 0.5⋅wby  0.5⋅gx - 0.5⋅wbx           0          -0
⎢                                                                             
⎢       0                  0                  0                  0            
⎢                                                                             
⎢       0                  0                  0                  0            
⎢                                                                             
⎣       0                  0                  0     

The above found transition matrix is a linearized continuous model but for EKF we need to 
discretize it. 
There are several methods available to discretize the continuous matrix. We will be apporaching 
one or two method and find out which one is easy to implement in C++.

The first of those method is Van-Loan method.

1. Van-Loan method for discretization

2. Series definition of a matrix exponential
    http://web.mit.edu/18.06/www/Spring17/Matrix-Exponentials.pdf

### Measurement model
The accelerometer measures the specific force $f_{b}$ at each time instance t and the accelerometer measurements are assumed to be corrupted by bias $b_{t}$ and noise $e_{t}.
 
$$
y_{a} = f_{b} + b_{t} + e_{t}
$$

The accelerometer noise is typically gaussian and hence for a properly calibrated sensor the noise covaraince matrix can be diagonal.

$$
a_{b} = i_{R_{n}} * g_{n} + i_{R_{n}} * \omega * V
$$

here 

$a_{b}$ is the acceleration experieced by the accelerometer in IMU coordinate frame.

$i_{R_{n}}$ is the rotation matrix that converts the values expressed in navigation coordinate frame to IMU frame.

$V$ is the velocity.

$\omega$ is the angular velocity expressed in navigation frame.

In [9]:
# The rotation matrix that converts the measurements expressed in navigation coordinate frame to IMU frame.
C_ned2b  = Matrix([[1-2*(qy**2+qz**2),   2*(qx*qy+qz*qw),    2*(qx*qz-qy*qw)],
                     [2*(qx*qy-qz*qw),   1-2*(qx**2+qz**2),    2*(qy*qz+qx*qw)], 
                     [2*(qx*qz+qy*qw),   2*(qy*qz-qx*qw),    1-2*(qx**2+qy**2)]])

Gravity is going to be 9.8m/sec2. Since we are not measureming velocity we ignore it for now.

In [10]:
gmps = symbols('g')
accel = Matrix([[0.0], [0.0], [gmps]])
zhat = C_ned2b * accel

In [11]:
H = zhat.jacobian(state)
print(H)
pprint(H)

Matrix([[-2*g*qy, 2*g*qz, -2*g*qw, 2*g*qx, 0, 0, 0], [2*g*qx, 2*g*qw, 2*g*qz, 2*g*qy, 0, 0, 0], [0, -4*g*qx, -4*g*qy, 0, 0, 0, 0]])
⎡-2⋅g⋅qy  2⋅g⋅qz   -2⋅g⋅qw  2⋅g⋅qx  0  0  0⎤
⎢                                          ⎥
⎢2⋅g⋅qx   2⋅g⋅qw   2⋅g⋅qz   2⋅g⋅qy  0  0  0⎥
⎢                                          ⎥
⎣   0     -4⋅g⋅qx  -4⋅g⋅qy    0     0  0  0⎦
